In [46]:
import numpy as np

from webbink import radius

## Calculating the ratio of bremsstrahlung to Compton cooling in a white dwarf boundary layer

We have claimed in the past that accretion disk boundary layers may be cooled by ambient radiation.  The code below allows us to calculate this under various physical conditions.  If electon cooling via bremsstrahlung is more efficient that cooling via inverse Compton scattering, then the radiation from the boundary layer will reflect the post-shock temperature of the gas in some way.  However, if Compton cooling is more efficient, then the maximum energy of electrons in the boundary layer will be decreased, and we don't expect to see hard X-rays characteristic of the post shock region.


Following Frank, King and Raine (2002), the ratio of bremsstrahlung to Compton lossed can be given by the equation:

$$\frac{t_{\rm Comp}}{t_{\rm cool}} = \frac{7.5 \times 10^{-5}N_{e}}{U_{\rm rad}T_{e}^{0.5}}$$ 

where $N_{e}$ and $T_{e}$ are the electron density and temperature in the boundary layer, and $U_{\rm rad}$ is the radiation energy density of the external radiation source that could be cooling the boundary layer (e.g. white dwarf surface, optically thick boundary layer).

In [173]:
def coolratio(nebl, urad, tshock):
    ratio = 7.5e-5 * nebl / (urad * tshock**0.5)
    return ratio

First, we'll set up some global variables that are needed in the various calculations below:


In [174]:
MSOL = 2e33 # mass of Sun, g
MU = 0.63 #mean molecular weight
MH = 1.67e-24 # mass of hydrogen, g
G = 6.67e-8
K = 1.38e-16 # Boltzmann Constant, erg/K
KTOKEV = 8.6173e-8 # convert Kelvin to keV
C = 2e10 # speed of light, cm/s
MDOTGS = 6.34e+25 # convert mass accretion rate in msol/yr to g/s

The maximum temperature $T_{e}$ of the post-shock flow is our key observable. We first want to calculate what this would be if the temperature is simply set by the gravitational potential of the white dwarf.

In [175]:
def tshock(mwd, unit='K'):
    '''Returns the shock temperature for the innermost Keplerian orbit of an
    accretion disk around a white dwarf of mass mwd in solar units.
    Units can be Kelvin ('K') or keV ('keV').'''
    # mass in solar masses
    rwd = radius(mwd)
    mwdg = mwd * MSOL # mass of white dwarf in grams
    tshock = 3.0 * MU * MH * G * mwdg / (16.0 * K * rwd)
    if unit.lower() == 'k':
        return tshock
    elif unit.lower() == 'kev':
        return tshock * KTOKEV
    else:
        print("Units must be Kelvin (K) or keV (keV).  Please check your units.")

Next, we need a way to calculate the energy density of the radiation field, $U_{\rm rad}$, that might cool the post shock region. This is agnostic about what the source of the radiation is: we just need a luminosity (which we'll calculate elsewhere).

We assume that the source of radiation covers some fraction frad of the surface of a white dwarf of mass mwd (in solar units), so that

$$U_{\rm rad} = \frac{L_{\rm rad}}{4 \pi R_{\rm WD}^{2} c},$$ where $c$ is the speed of light.

In [176]:
def urad(lrad, mwd, frad):
    '''Returns the radiation energy density for a radiation source of 
    luminosity Lrad (in erg/s) assuming it covers a fraction frad of
    the surface of a white dwarf of mass mwd.
    Mass should be in solar units.'''
    if frad <= 0 or frad > 1:
        print("frad must be between 0 and 1. Please check and try again.")
    else:
        rwd = radius(mwd)
        urad = lrad / (4.0 * np.pi * rwd ** 2.0 * frad * C)
        return urad

The other key parameter in determining the cooling ratio is the electron density in the boundary layer. Assuming a constant mass accretion rate through the boundary layer, we can calculate this as 
$$N_{e} = \frac{\dot{M}}{4 \pi R_{\rm WD}^{2} \mu m_{\rm H} f_{\rm acc} v_{\rm acc}},$$ where $f_{\rm acc}$ is the fraction of the white dwarf surface over which the accretion takes place, and $v_{\rm acc}$ is the radial velocity with which material is moving through the boundary layer.

In [177]:
def nebl(mwd, mdot, facc, vacc):
    '''Returns the density in the boundary layer for a given accretion 
    rate mdot, white dwarf mass mwd, accretion velocity vacc (the radial
    velocity of material through the boundary layer) and surface area
    fraction facc over which material accretes on to the white dwarf.
    Mdot should be in solar masses per year, mwd in solar units, and
    vacc in km/s.  Facc should be between >0 and <= 1.'''
    if facc <=0 or facc > 1:
        print("facc should be between >0 and <= 1. Please check and try again.")
    else:
        rwd = radius(mwd)
        mdot = mdot * MDOTGS
        vacc = vacc * 1e5
        nebl = mdot / (4 * np.pi * MU * MH * rwd**2 * facc * vacc)
        print("The electron density is {:03.1e} cm^-3.".format(nebl))
        return nebl

The accretion speed through the boundary layer is not well known. The innermost Keplerian velocity of the disk, $v_{\rm Kep}$, is given by

$$v_{\rm Kep} = \sqrt{\frac{G M_{\rm WD}}{R_{\rm WD}}}.$$

In [178]:
def vkep(mwd):
    rwd = radius(mwd)
    mwdg = mwd * MSOL
    vkep = (G * mwdg / rwd) ** 0.5
    return vkep

How much of this circular velocity gets transferred into radial velocity? Explore claims here.  For now, we assume that the accretion velocity, $v_{\rm acc}$ is low, of order 10 km s$^{-1}$.

In [179]:
vacc = 10

What about $f_{\rm acc}$, the fraction of the white dwarf surface over which the accretion takes place?  This depends on the assumed structure of the boundary layer, and whether it remains optically thick or thin.  Several approximations of this exist in the literature.  The scale height of the disk, $z$, depends on the accretion rate. Patterson & Raymond (1985) explore the scale height of the disk under a number of conditions, and come up with expressions in the optically thick (high-Mdot) and optically thin (low M-dot) regimes.  These have been coded up below. The conditions under which these are relevant are unclear.

We assume that accretion takes place over a band around the white dwarf of height 2$z$ and area 4$\pi z R_{\rm WD}$.  Since the surface area of the white dwarf is $4 \pi R_{\rm WD}^{2}$, then $f_{\rm acc}$ is given by:

$$f_{\rm acc} = \frac{z}{R_{\rm WD}}.$$




In [180]:
def facc(mwd, mdot, mode="thick"):
    '''Returns an estimte of the accretion fraction facc for a thick or thin
    boundary layer.  Facc is calculated as z/4r, where z is the scale height
    of disk.  If mode = "thick", the disk scale height is assumed to be equal 
    to the disk scale height. If mode = "thin", use approximation of Tylenda
    (1981).  See Patterson & Raymond, 1985 for details.'''
    if mode.lower() == "thick":
        z = (np.sqrt(radius(mwd)) ** 2.0 * \
            (6.96e-4 * (mwd/0.7) ** -0.85 * (mdot * MDOTGS / 1e18) ** -0.22 + \
            (7.29e-4 * (mwd/0.7) ** 0.8 * (mdot * MDOTGS / 1e18))))
        facc = z / radius(mwd)
        print("mode is {} and scale height z = {:03.1e} cm".format(mode, z))
        return facc
    elif mode.lower() == "thin":
        z = 3.85e8 * (mwd/0.7)**0.1 * (mdot * MDOTGS / 1e15)**-0.5
        facc = z / radius(mwd)
        print("mode is {} and scale height z = {:03.1e} cm".format(mode, z))
        return facc
    else:
        print("mode must be thick or thin")

### The boundary layer in RS Oph

Let's consider the case of RS Oph, as we explored in the 2011 paper.  First, we'll set up some parameters for this system. We'll assume that the X-rays trace *only* the accretion rate through the optically thin portion of the boundary layer:

In [181]:
mwd = 1.35
mdot = 2e-9
mode = "thin"

For white dwarf of this mass, the maximum post-shock temperature should be:

In [182]:
print("Shock temperature for {} Msol white dwarf is {:03.1e} K.".format(mwd, tshock(mwd, unit='K')))
print("Shock temperature for {} Msol white dwarf is {:03.1f} keV.".format(mwd, tshock(mwd, unit='keV')))

Shock temperature for 1.35 Msol white dwarf is 9.5e+08 K.
Shock temperature for 1.35 Msol white dwarf is 81.5 keV.


#### Cooling by the still hot white dwarf

First, let's consider the case that the still-hot, but not burning white dwarf surface is the source of Compton seed photons.  In this case, the luminosity of the radiation $L_{\rm rad}$ is given by

$$L_{\rm rad} = 4 \pi R_{\rm WD}^{2} \sigma T^{4},$$

where T is constrained to be less than 395,000K (Nelson et al. 2011).  Since the radiation is from the entire white dwarf surface, $f_{\rm rad}$ = 1.

In [183]:
lrad = 4 * np.pi * radius(mwd)**2 * 5.67e-5 * 395000**4.0
print("Lrad = {:03.1e}".format(lrad))
frad = 1.0

Lrad = 1.3e+36


The ratio of cooling via Compton scattering is:

In [184]:
ratio = coolratio(nebl(mwd, mdot, facc(mwd, mdot, mode="thin"), vacc),
                  urad(lrad, mwd, frad),
                  tshock(mwd, unit="K"))
print("Cooling ratio = {:03.1e}".format(ratio))

mode is thin and scale height z = 3.7e+07 cm
The electron density is 9.6e+17 cm^-3.
Cooling ratio = 3.4e+01


If the white dwarf has cooled to a lower temperature, say 100,000K, the ratio gets larger:

In [186]:
lrad = 4 * np.pi * radius(mwd)**2 * 5.67e-5 * 100000**4.0
print("Lrad = {:03.1e}".format(lrad))
ratio = coolratio(nebl(mwd, mdot, facc(mwd, mdot, mode=mode), vacc),
                  urad(lrad, mwd, frad),
                  tshock(mwd, unit="K"))
print("Cooling ratio = {:03.1e}".format(ratio))

Lrad = 5.3e+33
mode is thin and scale height z = 3.7e+07 cm
The electron density is 9.6e+17 cm^-3.
Cooling ratio = 8.3e+03



#### Cooling by an optically thick boundary layer

We assume here that a multizone boundary layer can exist, and that the optically thick portion would be towards the midplane.  In this case, the accretion luminosity will be given by

$$L_{\rm rad} = \frac{GM_{\rm WD} \dot{M}_{\rm thick}}{R_{\rm WD}}$$

where $\dot{M}_{\rm thick}$ is the accretion rate through the optically thick part of the boundary layer.


In [194]:
mdotthick = 3e-9

In [196]:
lrad = G * mwd * MSOL * mdotthick * MDOTGS / radius(mwd)
print("Lrad = {:03.1e}".format(lrad))
ratio = coolratio(nebl(mwd, mdot, facc(mwd, mdot, mode="thin"), vacc),
                  urad(lrad, mwd, facc(mwd, mdotthick, mode="thick")),
                  tshock(mwd, unit="K"))
print("Cooling ratio = {:03.1e}".format(ratio))

Lrad = 1.3e+35
mode is thin and scale height z = 3.7e+07 cm
The electron density is 9.6e+17 cm^-3.
mode is thick and scale height z = 2.2e+05 cm
Cooling ratio = 2.8e-01


### What about T CrB?

Let's see what should happen in the case of T CrB.

In [199]:
mwd = 1.2
mdot = 1e-9
mode = "thin"
mdotthick = 2e-9
lrad = G * mwd * MSOL * mdotthick * MDOTGS / radius(mwd)
print("Lrad = {:03.1e}".format(lrad))
ratio = coolratio(nebl(mwd, mdot, facc(mwd, mdot, mode="thin"), vacc),
                  urad(lrad, mwd, facc(mwd, mdotthick, mode="thick")),
                  tshock(mwd, unit="K"))
print("Cooling ratio = {:03.1e}".format(ratio))

Lrad = 4.9e+34
mode is thin and scale height z = 5.1e+07 cm
The electron density is 2.3e+17 cm^-3.
mode is thick and scale height z = 3.5e+05 cm
Cooling ratio = 5.4e-01
